Alex Kappes <br>
Problem Set 6 <br>
EconS 512

**Problem 1**

**(a)**

In [ ]:
import numpy as np
from numpy.linalg import inv
import pandas as pd

df = pd.read_csv('/home/akappes/WSU/512_MetricsII/merit_data.csv')

class estimate:
    def __init__(self, dep, indep, df):
        self.dep = np.asmatrix(dep).T
        self.indep = np.asmatrix(indep)
        self.df = df

    def beta(self):
        b = inv(self.indep.T * self.indep) * self.indep.T * self.dep
        return b

    def error(self, beta):
        resid = self.dep - self.indep * beta
        return resid

    def robust(self, e):
        k = self.indep.shape[1]
        n = self.indep.shape[0]

        s = np.zeros((k, k))
        i = 1

        while i < n:

            s = s + e[i, 0]**2 * self.indep[i].T * self.indep[i]
            i = i + 1

            if i > n:
                break

        w_hce = inv(self.indep.T * self.indep) * s * inv(self.indep.T * self.indep)
        w_se = np.sqrt(np.diag(w_hce))
        return w_se

    def se_cluster_s(self, g, e):
        group_s = pd.DataFrame(df[g].unique().tolist()).rename(columns={0: 'group'})

        l_s = {}
        for state in group_s['group']:
            l_s[state] = pd.DataFrame()

        for key in l_s:
            l_s[key] = df[df[g] == key].index.tolist()

        group_e = pd.DataFrame(e).rename(columns={0: 'ehat'})

        d_sigsq = {}
        for state in group_s['group']:
            d_sigsq[state] = pd.DataFrame()

        for key in d_sigsq:
            d_sigsq[key] = (np.asmatrix(group_e.loc[l_s[key]]).T *
                            np.asmatrix(group_e.loc[l_s[key]])) / len(l_s[key])
        
        sigsq_df = pd.DataFrame({'sigsq': 0}, index=df.index)

        for key in d_sigsq:
            sigsq_df.loc[df[df[g] == key].index.tolist(), 'sigsq'] = d_sigsq[key][0, 0]

        sigsq_l = np.array(sigsq_df)
        s = np.multiply(np.divide(1, sigsq_l), np.eye(len(self.indep)))
        covmat_s = inv(self.indep.T * s * self.indep)
        return np.sqrt(np.diag(covmat_s))
        
        # code commented out below represents Greene's (8ed) equation (9-33) for clustered covmat estimator.
        # He specifies by grouping and summing over all groups but this produces singular x_g'x_g 
        # matrices for all of this binary fixed effect data. I've created an identity matrix with 
        # repeated 1 / sighat_g^2 values corresponding to index values in X for groupings.
        # Each within group obs is weighted appropriately and det(X) exists, providing 
        # a computable clustered covmat estimator. Code below is kept for future reference
        # when not working with bullshit class data.
        
        # xtxi = {}
        # for state in group_s['group']:
        #     xtxi[state] = pd.DataFrame()
        #
        # for key in xtxi:
        #     xtxi[key] = inv(np.asmatrix(df[df[g] == key][x_vars]).T * np.asmatrix(df[df[g] == key][x_vars]))
        #
        # covmat_s_g = {}
        # for state in group_s['group']:
        #     covmat_s_g[state] = pd.DataFrame()
        #
        # for key in covmat_s_g:
        #     covmat_s_g[key] = d_sigsq[key] * xtxi[key]
        #
        # covmat_l = [covmat_s_g[group_s.loc[0, 'group']]]
        # i = 1
        # while i < len(group_s):
        #     covmat_l.append(covmat_s_g[group_s.loc[i, 'group']])
        #     i += 1
        #     if i > len(group_s):
        #         break
        #
        # covmat_s = sum(covmat_l)
        # return np.sqrt(np.diag(covmat_s))

# data construction
bin_state = pd.DataFrame(0, columns=df['state'].unique().tolist()[:-1], index=df.index)
bin_year = pd.DataFrame(0, columns=df['year'].unique().tolist()[:-1], index=df.index)

for j in bin_state.columns:
    bin_state.loc[df[df['state'] == j].index.tolist(), j] = 1

for j in bin_year.columns:
    bin_year.loc[df[df['year'] == j].index.tolist(), j] = 1

df.insert(loc=0, column='ones', value=1)
df = pd.concat([df, bin_state, bin_year], axis=1)

X = df.drop(['coll', 'year', 'state', 'chst'], axis=1)
x_vars = X.columns
y = df['coll']

# estimation
est = estimate(y, X, df)
params = est.beta()
resids = est.error(params)
whites_hce = est.robust(resids)
clustered_state_se = est.se_cluster_s('state', resids)